In [2]:
import matplotlib.pyplot as plt
import cloudpickle
import numpy as np

path = "output/flex/results_1.pkl"
outputfolder = "output/flex/"

with open(path, "rb") as file:
    results = cloudpickle.load(file)

results = list(results)
for rollout in results:
    for i in range(1,15):
        aid = f"H{i}"
        agent_actions = []
        agent_demand = []
        agent_charge = []
        agent_supply = []
        agent_prod = []
        prices = []
        agent_actions += list(rollout.actions_for_agent(aid))
        agent_demand += list(rollout.metrics[f"{aid}/current_load"])
        agent_charge += list(rollout.metrics[f"{aid}/current_charge"])
        agent_supply += list(rollout.metrics[f"{aid}/current_supply"])
        agent_prod += list(rollout.metrics[f"{aid}/current_prod"])
        prices += list(rollout.metrics["env/current_price"])

        for idx, action in enumerate(agent_actions):
            if action is None:
                agent_actions[idx] = agent_actions[idx + 1] if idx + 1 < len(agent_actions) else 6

        start = 48*30*1 
        end = 48*30*1 + 48*2

        agent_actions = agent_actions[start:end:2]
        agent_demand = agent_demand[start:end:2]
        agent_charge = agent_charge[start:end:2]
        agent_supply = agent_supply[start:end:2]
        agent_prod = agent_prod[start:end:2]
        prices = prices[start:end:2]
    

        fig, ax1 = plt.subplots()

        ax2 = ax1.twinx()
        ax2.scatter(range(len(agent_actions)), agent_actions, label="Action", color='tab:green', alpha=0.5)
        ax2.set_ylabel('Action')
        ax2.set_yticks([0, 1, 2, 3, 4, 5])
        ax2.set_yticklabels(['buy', 'buycharge', 'sell', 'sellcharge', 'charge', 'noop'])
        ax2.legend(loc='upper right')
        
        #ax1.bar(range(len(agent_demand)), agent_demand, label="Demand", color='tab:blue', alpha=1)
        ax1.bar(range(len(agent_charge)), agent_charge, label="Charge", color='tab:orange', alpha=0.5)
        ax1.bar(range(len(agent_supply)), agent_supply, label="Supply", color='tab:red', alpha=0.6)
        #ax1.bar(range(len(agent_prod)), agent_prod, label="Production", color='tab:purple', alpha=0.6)
        ax1.scatter(range(len(prices)), prices, label="Price", color='tab:grey', alpha=0.8)
        ax1.set_xlabel('Time step')
        ax1.set_ylabel('Demand/Charge')
        ax1.legend(loc='upper left')


        fig.tight_layout()
        plt.savefig(f"{outputfolder}demand_charge_action_{aid}.png")
        plt.close()




        # Plot distribution of agent action per step for all rollouts
        # plt.hist(agent_actions, bins=np.arange(-0.5, 6, 1), edgecolor='black')
        # plt.xticks(np.arange(6))
        # plt.title("Distribution of action values")
        # plt.xlabel("Agent action")
        # plt.ylabel("Frequency")
        # plt.savefig(f"action_dist_{aid}.png")
        # plt.close()
